In [1]:
# Import GBApy
#import gba

import os
import sys
sys.path.append('../gba/')
import GbaModel as gba

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
model = gba.read_csv_model("efm2", "../toy_models")
model.summary()

❯ Info,KM backward constants not loaded.


❯ Info,KA constants not loaded.


❯ Info,KI constants not loaded.


❯ Info,Constant reactions not loaded.


❯ Info,Protein contributions not loaded.


❯ Info,LP solution not loaded.


Category,Count
Nb metabolites,6
Nb external metabolites,2
Nb internal metabolites,4
Category,Count
Nb reactions,5
Nb exchange reactions,2
Nb internal reactions,2
Category,Count
Column rank,4
Is full column rank?,False


In [3]:
help(model.add_condition)
model.add_condition("2", rho=340.0, metabolites={"x_C1": 0.01, "x_C2": 0.1})

Help on method add_condition in module GbaModel:

add_condition(condition_id: int, rho: float, default_concentration: Optional[float] = 1.0, metabolites: Optional[dict[str, float]] = None) -> None method of GbaModel.GbaModel instance
    Add an external condition to the GBA model.

    Parameters
    ----------
    condition_id : int
        Identifier of the condition.
    rho : float
        Total density of the cell (g/L).
    default_concentration : float
        Default concentration of metabolites (g/L).
    metabolites : dict[str, float]
        Dictionary of metabolite concentrations (g/L).



In [4]:
model.solve_local_linear_problem()
model.LP_solution

array([2.50000001e-02, 9.75000000e-01, 1.00000000e-10, 9.50000000e-01,
       9.25000000e-01])

In [5]:
model.set_f0(model.LP_solution)
model.clear_all_trajectories()
model.gradient_ascent(condition_id="1", track=True, verbose=True, max_time=14.442456859579531, print_period=500)

❯ Iteration: 500 (time = 3.5384000000000224, mu = 0.10127732000942281, dt = 0.0064).

❯ Iteration: 1000 (time = 7.78774399999997, mu = 0.11871351561094054, dt = 0.008192000000000001).

❯ Iteration: 1500 (time = 10.7915865600001, mu = 0.1438971870366064, dt = 0.005242880000000001).

❯ Iteration: 2000 (time = 13.400233932800175, mu = 0.2129046102730536, dt = 0.006710886400000002).

❯ Iteration: 2500 (time = 14.29033369264138, mu = 0.3640970109913081, dt = 0.00017179869184000003).

❯ Iteration: 3000 (time = 14.442439610544945, mu = 0.5952653682078054, dt = 5.629499534213121e-11).

❯ Iteration: 3500 (time = 14.442439912454939, mu = 0.5952695861958643, dt = 1.8014398509481987e-09).

❯ Iteration: 4000 (time = 14.442449573576894, mu = 0.595404614509646, dt = 5.764607523034236e-08).

❯ Gradient ascent: maximum time reached (condition=1, mu=0.59544, nb iterations=4499, nb fixed=4473).

(False, 4.887880325317383)

In [6]:
fig = model.create_figure("Mu trajectory")
model.add_trajectory(fig, source="GA", x_var="iter", y_var="f_rxn1", name="Rxn1")
model.add_trajectory(fig, source="GA", x_var="iter", y_var="f_rxn2", name="Rxn2")
model.add_trajectory(fig, source="GA", x_var="iter", y_var="mu", name="Mu")
fig.show()

In [7]:
help(model.gradient_ascent)
model.GA_tracker

Help on method gradient_ascent in module GbaModel:

gradient_ascent(condition_id: Optional[str] = '1', max_time: Optional[float] = 10.0, initial_dt: Optional[float] = 0.01, track: Optional[bool] = False, variables: Optional[list[str]] = ['f'], label: Optional[int] = 1, verbose: Optional[bool] = False, print_period: Optional[int] = 0) -> tuple[bool, float] method of GbaModel.GbaModel instance
    Run a gradient ascent algorithm to find the optimal flux state.

    Parameters
    ----------
    condition_id : str, default="1"
        Condition identifier.
    max_time : float, default=10.0
        Maximum time for the algorithm.
    initial_dt : float, default=0.01
        Initial time step.
    track : bool, default=False
        Track the trajectory of variables.
    variables : list[str], default=["f"]
        Additional variables to track.
    label : int, default=1
        Label for the trajectory.
    verbose : bool, default=False
        Verbose mode.
    print_period : int, defau

,label,condition,iter,dt,t,mu,fixed,f_rxn1,f_rxn2,f_rxn3,f_rxn4,f_Ribosome
0,1,1,0,0.01,0.0,0.089088,0,0.025,0.975,0.0,0.95,0.925
1,1,1,1,0.01,0.01,0.08923,1,0.025734,0.974266,0.000672,0.950633,0.925245
2,1,1,2,0.01,0.02,0.089362,2,0.026465,0.973535,0.001291,0.95121,0.925543
3,1,1,3,0.01,0.03,0.089487,3,0.027193,0.972807,0.001871,0.951741,0.925883
4,1,1,4,0.01,0.04,0.089607,4,0.027917,0.972083,0.00242,0.952235,0.926255
...,...,...,...,...,...,...,...,...,...,...,...,...
4469,1,1,4495,0.0,14.442457,0.595442,4469,1.0,0.0,0.940679,0.0,0.897322
4470,1,1,4496,0.0,14.442457,0.595442,4470,1.0,0.0,0.940679,0.0,0.897322
4471,1,1,4497,0.0,14.442457,0.595442,4471,1.0,0.0,0.940679,0.0,0.897322
4472,1,1,4498,0.0,14.442457,0.595442,4472,1.0,0.0,0.940679,0.0,0.897322


In [8]:
model.f

array([9.99999943e-01, 5.70181216e-08, 9.40679091e-01, 1.00000000e-10,
       8.97321716e-01])

In [9]:
model.set_f0(model.f)
model.gradient_ascent(condition_id="2", track=True, verbose=True, max_time=14.649451775311244, print_period=500, label=2)

❯ Iteration: 500 (time = 3.820000000000019, mu = 0.10149981024315717, dt = 0.0064).

❯ Iteration: 1000 (time = 8.01942399999997, mu = 0.11885354949921934, dt = 0.008192000000000001).

❯ Iteration: 1500 (time = 11.04423808000005, mu = 0.1444717416592552, dt = 0.005242880000000001).

❯ Iteration: 2000 (time = 13.33579607040011, mu = 0.19838896551016436, dt = 0.003355443200000001).

❯ Iteration: 2500 (time = 14.485370910720071, mu = 0.3574406774803352, dt = 0.00042949672960000007).

❯ Iteration: 3000 (time = 14.649451424100882, mu = 0.5952601879690298, dt = 7.036874417766401e-11).

❯ Gradient ascent: maximum time reached (condition=2, mu=0.59527, nb iterations=3499, nb fixed=3477).

(False, 4.803580045700073)

In [10]:
model.GA_tracker

,label,condition,iter,dt,t,mu,fixed,f_rxn1,f_rxn2,f_rxn3,f_rxn4,f_Ribosome
0,1,1,0,0.01,0.0,0.089088,0,0.025,0.975,0.0,0.95,0.925
1,1,1,1,0.01,0.01,0.08923,1,0.025734,0.974266,0.000672,0.950633,0.925245
2,1,1,2,0.01,0.02,0.089362,2,0.026465,0.973535,0.001291,0.95121,0.925543
3,1,1,3,0.01,0.03,0.089487,3,0.027193,0.972807,0.001871,0.951741,0.925883
4,1,1,4,0.01,0.04,0.089607,4,0.027917,0.972083,0.00242,0.952235,0.926255
...,...,...,...,...,...,...,...,...,...,...,...,...
7947,2,2,3495,0.0,14.649452,0.595265,3473,0.000041,0.999959,0.0,0.94065,0.897327
7948,2,2,3496,0.0,14.649452,0.595265,3474,0.000041,0.999959,0.0,0.94065,0.897327
7949,2,2,3497,0.0,14.649452,0.595265,3475,0.000041,0.999959,0.0,0.94065,0.897327
7950,2,2,3498,0.0,14.649452,0.595265,3476,0.000041,0.999959,0.0,0.94065,0.897327


In [11]:
fig = model.create_figure("Mu trajectory")
model.add_trajectory(fig, source="data", x_var="iter", y_var="f_rxn1", name="Rxn1", data=model.GA_tracker[model.GA_tracker["label"] == 1])
model.add_trajectory(fig, source="data", x_var="iter", y_var="f_rxn2", name="Rxn2", data=model.GA_tracker[model.GA_tracker["label"] == 1])
model.add_trajectory(fig, source="data", x_var="iter", y_var="mu", name="Mu", data=model.GA_tracker[model.GA_tracker["label"] == 1])
fig.show()

In [12]:
fig = model.create_figure("Mu trajectory")
fig.add_trace(go.Scatter(x=model.GA_tracker[model.GA_tracker["label"] == 1]["iter"], y=model.GA_tracker[model.GA_tracker["label"] == 1]["f_rxn1"], mode="lines", name="Rxn1"))
fig.add_trace(go.Scatter(x=model.GA_tracker[model.GA_tracker["label"] == 2]["iter"]+4499, y=model.GA_tracker[model.GA_tracker["label"] == 2]["f_rxn1"], mode="lines", name="Rxn1"))
fig.show()